In [1]:
from typing import TYPE_CHECKING


if TYPE_CHECKING:
    from math_rag.application.containers import ApplicationContainer
    from math_rag.infrastructure.containers import InfrastructureContainer

    application_container: ApplicationContainer
    infrastructure_container: InfrastructureContainer

In [ ]:
RESET = False
%load_ext hooks.notebook_hook

2025-06-23 20:54:19,085 - INFO - datasets - config.py:54 - PyTorch version 2.6.0 available.


In [ ]:
from pathlib import Path


google_drive_repository = infrastructure_container.google_drive_repository()
math_article_parser_service = infrastructure_container.math_article_parser_service()

file_id = google_drive_repository.get_file_id(
    Path('ml/lectures/L07-LogisticRegression2/2024_08_10_2174b40686820b4cb591g.tex')
)

if not file_id:
    raise ValueError()

file_content = google_drive_repository.get_file_by_id(file_id)

In [ ]:
from math_rag.core.models import MathArticle, MathExpression


math_article = MathArticle(
    math_expression_dataset_id=None,
    index_id=None,
    name='article',
    bytes=file_content.getvalue(),
)
math_nodes_, _, template = math_article_parser_service.parse_for_index(math_article)
math_expressions: list[MathExpression] = ...  # NOTE obtained from math_nodes_
index_to_katex = {i: math_expression.katex for i, math_expression in enumerate(math_expressions)}

In [ ]:
from math_rag.infrastructure.utils import TemplateContextChunkerUtil, TemplateFormatterUtil


template_chunks = TemplateContextChunkerUtil.chunk(template, max_context_size=1000)
chunks = [
    TemplateFormatterUtil.format(chunk, index_to_katex, omit_wrapper=False)
    for chunk in template_chunks
]

# TODO send to llm
# 1. MathExpressionDescriptionWriter
# 2. MathExpressionDescriptionRefiner
# 3. MathExpressionLinker?? -> checks whether 2 descriptions are actually the same (embedding clustering before)
#


class MathExpressionDescription:
    pass